# Transit Corridor Recs - Benefit from Marginal Improvements

## Brainstorm

Filter to intersecting SHN or other route (all transit routes except for those that run on SHN) within a district if:
* average route speed is above **12 mph**  **AND**
* 50%+ of its trips are competitive against a car traveling the same path (competitive is 1.5x...so bus can take 50% more time)
* Get speedmaps for these set of routes for more detailed info?
* Notes: Mostly D7 if we raise the bar too high, but let's at least get D10 Stockton to show up.
    * D7: Rail shows up mostly, Expo, Gold, Red, Green are all competitive 
    * Loosen constraints so more buses show up

In [1]:
import branca 
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import Markdown

from shared_utils import calitp_color_palette as cp
from shared_utils import geography_utils

catalog = intake.open_catalog("./*.yml")

CUSTOM_CATEGORICAL = [
    cp.CALITP_CATEGORY_BRIGHT_COLORS[2], # yellow
    cp.CALITP_CATEGORY_BRIGHT_COLORS[1], # orange
    cp.CALITP_CATEGORY_BRIGHT_COLORS[0], # blue
    cp.CALITP_CATEGORY_BRIGHT_COLORS[5], # purple  
    cp.CALITP_CATEGORY_BRIGHT_COLORS[3], # green
]

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
gdf = catalog.bus_routes_all_aggregated_stats.read().to_crs(geography_utils.WGS84)

include = ["intersects_shn", "other"]
gdf = gdf[gdf.category.isin(include)].reset_index(drop=True)

In [3]:
METRIC_CUTOFFS = {
    # show routes with average speeds >= 12 mph
    "mean_speed_mph": 12, 
    # show routes with >= 10% of trips being competitive against a car 
    # competitive is if it takes bus no longer than +20% longer than a car
    "pct_trips_competitive": 0.50,
}

In [4]:
def set_cutoffs(gdf: gpd.GeoDataFrame, 
                speed_threshold: int, 
                pct_competitive_threshold: float,
               ) -> gpd.GeoDataFrame: 
    
    print(f"speed cutoff: {speed_threshold}, pct competitive cutoff: {pct_competitive_threshold}")
    # Get statewide counts to see how many fall into each district
    subset = gdf[(gdf.mean_speed_mph >= speed_threshold) &
                 (gdf.pct_trips_competitive >= pct_competitive_threshold)
                ]
    print(f"# obs statewide: {len(subset)}")
    display(subset.caltrans_district.value_counts())

In [5]:
# Mostly D7 if we raise the bar too high
# At least get D10 Stockton to show up
for speeds in [12, 15, 18]:
    for pct_competitive in [0.4, 0.5, 0.6]:
        set_cutoffs(gdf, speeds, pct_competitive)

speed cutoff: 12, pct competitive cutoff: 0.4
# obs statewide: 24


07 - Los Angeles    14
04 - Oakland         9
10 - Stockton        1
Name: caltrans_district, dtype: int64

speed cutoff: 12, pct competitive cutoff: 0.5
# obs statewide: 23


07 - Los Angeles    13
04 - Oakland         9
10 - Stockton        1
Name: caltrans_district, dtype: int64

speed cutoff: 12, pct competitive cutoff: 0.6
# obs statewide: 18


...

07 - Los Angeles    7
Name: caltrans_district, dtype: int64

speed cutoff: 18, pct competitive cutoff: 0.5
# obs statewide: 6


07 - Los Angeles    6
Name: caltrans_district, dtype: int64

speed cutoff: 18, pct competitive cutoff: 0.6
# obs statewide: 5


07 - Los Angeles    5
Name: caltrans_district, dtype: int64

In [6]:
def subset_to_district_sort_metric(gdf: gpd.GeoDataFrame, 
                                   district: str,
                                   speed_cutoff: int = 12,
                                   pct_competitive_cutoff: float = 0.05
                                  ): 
    keep_cols = ["calitp_itp_id", "route_id", 
            "mean_speed_mph", "pct_trips_competitive",
            "category", "geometry"]
    
    df = (gdf[(gdf.caltrans_district == district) & 
              (gdf.mean_speed_mph >= speed_cutoff) & 
              (gdf.pct_trips_competitive >= pct_competitive_cutoff)
             ]
          .reset_index(drop=True)
          [keep_cols]
         )
    
    df2 = df.assign(
        mean_speed_mph = df.mean_speed_mph.round(2),
        pct_trips_competitive = df.pct_trips_competitive.round(3)
    )
               
    return df2

In [7]:
TILES = "CartoDB Positron" 
def make_map(gdf: gpd.GeoDataFrame, district: str):        
    m = gdf.explore("mean_speed_mph", categorical = False, 
                    cmap = ZERO_THIRTY_COLORSCALE, tiles = TILES)
    
    display(m)
    
    m2 = gdf.explore("route_id", categorical = True, 
                    cmap = "tab20", tiles = TILES)
    
    display(m2)
    
    display(gdf.sort_values(
        ["mean_speed_mph", "pct_trips_competitive"], 
        ascending=[True, True]).drop(columns = ["geometry", "category"]))

In [8]:
districts = gdf[gdf.caltrans_district.notna()].caltrans_district.unique().tolist()

for d in sorted(districts):
    display(Markdown(f"### District {d}"))

    subset = subset_to_district_sort_metric(
        gdf, d, 
        speed_cutoff = METRIC_CUTOFFS["mean_speed_mph"],
        pct_competitive_cutoff = METRIC_CUTOFFS["pct_trips_competitive"]
    )

    if len(subset) > 0:
        make_map(subset, d)
    else:
        print("No routes meet this criteria")

### District 01 - Eureka

No routes meet this criteria


### District 02 - Redding

No routes meet this criteria


### District 03 - Marysville

...

,calitp_itp_id,route_id,mean_speed_mph,pct_trips_competitive
0,284,380,15.89,0.5


### District 11 - San Diego

No routes meet this criteria


### District 12 - Irvine

No routes meet this criteria
